In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
data = pd.read_csv("data.csv")
labels = pd.read_csv("labels.csv")

In [ ]:
data.head()

In [ ]:
labels.head()

In [ ]:
#merge both the datasets

labeled_data = data.merge(labels)
labeled_data.head()

In [ ]:
labeled_data.shape

In [ ]:
#plot as hierarchically-clustered heatmap

columns = [column for column in labeled_data.columns if labeled_data[column].dtype == 'float64']

sns.clustermap(labeled_data[columns], figsize=(9,9))

In [ ]:
def all_v(p):
    for x in range(0, len(p)):
        if p[x] < 0.05:
            return 0
        elif x >= len(p) - 1:
            return 1

In [ ]:
#Null Hypothesis Testing using TTest and ANOVA
import scipy.stats as stats

from scipy.stats import ttest_ind, f_oneway

gene_brca = labeled_data[labeled_data['Class'] == 'BRCA']
gene_kirc = labeled_data[labeled_data['Class'] == 'KIRC']
gene_coad = labeled_data[labeled_data['Class'] == 'COAD']
gene_luad = labeled_data[labeled_data['Class'] == 'LUAD']
gene_prad = labeled_data[labeled_data['Class'] == 'PRAD']

_, p_value_brca = ttest_ind(gene_brca[columns], labeled_data[columns].drop(gene_brca.index))
_, p_value_kirc = ttest_ind(gene_kirc[columns], labeled_data[columns].drop(gene_kirc.index))
_, p_value_coad = ttest_ind(gene_coad[columns], labeled_data[columns].drop(gene_coad.index))
_, p_value_luad = ttest_ind(gene_luad[columns], labeled_data[columns].drop(gene_luad.index))
_, p_value_prad = ttest_ind(gene_prad[columns], labeled_data[columns].drop(gene_prad.index))

_, p_value_anova = f_oneway(gene_brca[columns], gene_kirc[columns], gene_coad[columns], gene_luad[columns], gene_prad[columns])

if all_v(p_value_anova) < 0.05:
    print("Null hypothesis rejected. Significant differences among cancer types.")
else:
    print("Null hypothesis accepted. No significant differences among cancer types.")
    
if all_v(p_value_brca) < 0.05:
    print("Null hypothesis rejected. There are significant differences among BRCA and other cancer types.")
else:
    print("Null hypothesis accepted. There are no significant differences among BRCA and other cancer types.")

if all_v(p_value_kirc) < 0.05:
    print("Null hypothesis rejected. There are significant differences among KIRC and other cancer types.")
else:
    print("Null hypothesis accepted. There are no significant differences among KIRC and other cancer types.")
    
if all_v(p_value_coad) < 0.05:
    print("Null hypothesis rejected. There are significant differences among COAD and other cancer types.")
else:
    print("Null hypothesis accepted. There are no significant differences among COAD and other cancer types.")
    
if all_v(p_value_luad) < 0.05:
    print("Null hypothesis rejected. There are significant differences among LUAD and other cancer types.")
else:
    print("Null hypothesis accepted. There are no significant differences among LUAD and other cancer types.")
    
if all_v(p_value_prad) < 0.05:
    print("Null hypothesis rejected. There are significant differences among PRAD and other cancer types.")
else:
    print("Null hypothesis accepted. There are no significant differences among PRAD and other cancer types.")

In [ ]:
labeled_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = labeled_data[columns].values
Y = labeled_data['Class'].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [ ]:
#Dimensionality reduction

#PCA for samples

pca = PCA(n_components=25)
#pca.fit(labeled_data[columns])
pca.fit(X_train)

cluster_pca_data = pca.transform(labeled_data[columns])
pca_data = pca.transform(X_train)
test_pca_data = pca.transform(X_test)
print(pca_data)

In [ ]:
sns.clustermap(pca_data, figsize=(9,9))

In [ ]:
genes = labeled_data.T
genes = genes.drop(['Unnamed: 0'])
genes = genes.drop(['Class'])
genes.tail()

In [ ]:
#PCA for genes

pca = PCA(n_components=25)
pca.fit(genes)

pca_genes = pca.transform(genes)
print(pca_genes)

In [ ]:
#LDA

lda = LDA(n_components=2)
lda.fit(X_train, y_train)

lda_data = lda.transform(X_train)
test_lda_data = lda.transform(X_test)
print(lda_data)

In [ ]:
#t-SNE

tsne = TSNE(n_components=2)
tsne_data = tsne.fit_transform(labeled_data[columns])
print(tsne_data)

In [ ]:
#Clustering 
#Kmeans

#Clustering on all samples
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(cluster_pca_data)
predictions = kmeans.fit_predict(cluster_pca_data)
labeled_data.insert(len(labeled_data.columns), "Sample_Kmeans" , predictions, allow_duplicates=True)
labeled_data.head()

In [ ]:
plt.scatter(cluster_pca_data[:,0],cluster_pca_data[:,1], c=predictions)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('K-means Clustering on Sample PCA-transformed Data')
    
plt.show()

In [ ]:
#Clustering on all genes

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(pca_genes)
predictions = kmeans.fit_predict(pca_genes)
genes.insert(len(genes.columns), "Gene_Kmeans" , predictions, allow_duplicates=True)
genes.head()

In [ ]:
plt.scatter(pca_genes[:,0],pca_genes[:,1], c=predictions)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('K-means Clustering on Gene PCA-transformed Data')
plt.show()

In [ ]:
#Hierarchical Clustering

#Clustering on all samples

hc = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
hc.fit(cluster_pca_data)
h_predictions = hc.fit_predict(cluster_pca_data)
labeled_data.insert(len(labeled_data.columns), "Sample_HC" , h_predictions, allow_duplicates=True)
labeled_data.head(10)

In [ ]:
#Clustering on all genes
hc = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
hc.fit(pca_genes)
h_predictions = hc.fit_predict(pca_genes)
genes.insert(len(genes.columns), "Gene_HC" , h_predictions, allow_duplicates=True)
genes.head(10)

In [ ]:
#Mean-shit Clustering

#Clustering on all samples

meanshift = MeanShift()
meanshift.fit(cluster_pca_data)
m_predictions = meanshift.fit_predict(cluster_pca_data)
labeled_data.insert(len(labeled_data.columns), "Sample_MSC" , m_predictions, allow_duplicates=True)
labeled_data.head(10)

In [ ]:
#Clustering on all genes

#Takes too long

#meanshift = MeanShift()
#meanshift.fit(pca_genes)
#m_predictions = meanshift.fit_predict(pca_genes)
#genes.insert(len(genes.columns), "Sample_MSC" , m_predictions, allow_duplicates=True)
#genes.head()

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labeled_data['Class'].values)

In [ ]:
#Building Robust Classification Model to identify each type of Cancer

#from sklearn.model_selection import train_test_split

X = labeled_data[columns].values
Y = labeled_data['Class'].values

nX_train, nX_test, ny_train, ny_test = train_test_split(X, y_encoded, random_state=42)

In [ ]:
#Deep Neural Network

nn_model = Sequential()
nn_model.add(Dense(512, activation='relu', input_shape=(20531,)))
nn_model.add(Dropout(0,3))
nn_model.add(Dense(512, activation='relu'))
nn_model.add(Dropout(0,3))
nn_model.add(Dense(512, activation='relu'))
nn_model.add(Dropout(0,3))
nn_model.add(Dense(5, activation='softmax'))

In [ ]:
nn_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
nn_model.fit(nX_train, ny_train, epochs=5, validation_data=(nX_test, ny_test))

In [ ]:
from sklearn.metrics import classification_report

score = nn_model.evaluate(nX_test, ny_test)

In [ ]:
predictions = nn_model.predict(nX_test)
pred_class = np.argmax(predictions, axis=1)
print(pred_class)

In [ ]:
print(classification_report(ny_test, pred_class))

In [ ]:
#Random Forest

#rX_train, rX_test, ry_train, ry_test = train_test_split(pca_data, Y, random_state=42)

rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(pca_data, y_train)

rf_predictions = rf.predict(test_pca_data)
print(classification_report(y_test, rf_predictions))

In [ ]:
#multiclass SVM

svm = SVC()
svm.fit(pca_data, y_train)

svm_predictions = svm.predict(test_pca_data)
print(classification_report(y_test, svm_predictions))

In [ ]:
#Feature Selection Algorithms

#For the Deep Neural Network

#Deep Neural Networks don't rely on Feature Selection, because the number of layers and the 
#weights of the neurons at each level perform feature selection during the training phase


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

#For the Random Forest Model

rf_forward = RandomForestClassifier(max_depth=2, random_state=0)

forward_selector = SequentialFeatureSelector(rf_forward, direction='forward')
forward_selector.fit(pca_data, y_train)

rX_train_forward = forward_selector.transform(pca_data)
rX_test_forward = forward_selector.transform(test_pca_data)

rf.fit(rX_train_forward, y_train)

rf_predictions = rf.predict(rX_test_forward)
print(classification_report(y_test, rf_predictions))

In [ ]:
from sklearn.feature_selection import RFE

rf_backward = RandomForestClassifier(max_depth=2, random_state=0)

backward_selector = RFE(rf_backward)
backward_selector.fit(pca_data, y_train)

rX_train_backward = backward_selector.transform(pca_data)
rX_test_backward = backward_selector.transform(test_pca_data)

rf.fit(rX_train_backward, y_train)

back_rf_predictions = rf.predict(rX_test_backward)
print(classification_report(y_test, back_rf_predictions))

In [ ]:
#For the Multiclass SVM Model

svc_forward = SVC()

svc_forward_selector = SequentialFeatureSelector(svc_forward, direction='forward')
svc_forward_selector.fit(pca_data, y_train)

sX_train_forward = svc_forward_selector.transform(pca_data)
sX_test_forward = svc_forward_selector.transform(test_pca_data)

svm.fit(sX_train_forward, y_train)

svc_predictions = svm.predict(sX_test_forward)
print(classification_report(y_test, svc_predictions))

#backward elimination isn't used in multiclass SVM, feature selection comes as a result of overall performance

In [ ]:
rf_forward_data = np.concatenate((rX_train_forward, rX_test_forward), axis=0)
rf_backward_data = np.concatenate((rX_train_backward, rX_test_backward), axis=0)   
svc_forward_data = np.concatenate((sX_train_forward, sX_test_forward), axis=0)

print(rf_forward_data.shape)

In [ ]:
rff_df = pd.DataFrame(rf_forward_data)
rff_df.insert(len(rff_df.columns), "Class" , np.concatenate((y_train, y_test), axis=0), allow_duplicates=True)

rfb_df = pd.DataFrame(rf_backward_data)
rfb_df.insert(len(rfb_df.columns), "Class" , np.concatenate((y_train, y_test), axis=0), allow_duplicates=True)

svcf_df = pd.DataFrame(svc_forward_data)
svcf_df.insert(len(svcf_df.columns), "Class" , np.concatenate((y_train, y_test), axis=0), allow_duplicates=True)

gene_names = [column for column in rff_df.columns if rff_df[column].dtype == 'float64']

In [ ]:
rff_df.head()

In [ ]:
print(rff_df[gene_names].columns)

In [ ]:
#Statistical Significance Testing to validate genes from Feature Selection Step

from scipy.stats import ttest_ind
from scipy.stats import f_oneway

#Random Forest Values Forward
#t-test for one vs all

classes = ['PRAD', 'LUAD', 'BRCA', 'KIRC', 'COAD']
ttest_pvals = []

for x in rff_df[gene_names].columns:
    gene = rff_df[x]
    gene_p_val = []
    for y in classes:
        group1_expression = rff_df[gene_names[x]][rff_df['Class'] == y] 
        group2_expression = rff_df[gene_names[x]][rff_df['Class'] != y]  
        _, p_value = ttest_ind(group1_expression, group2_expression)
        gene_p_val.append(p_value)
    ttest_pvals.append(gene_p_val)

#F-test

_, p_value_f = f_oneway(rff_df[gene_names][rff_df['Class'] == 'PRAD'], rff_df[gene_names][rff_df['Class'] == 'LUAD'], rff_df[gene_names][rff_df['Class'] == 'BRCA'], rff_df[gene_names][rff_df['Class'] == 'KIRC'], rff_df[gene_names][rff_df['Class'] == 'COAD'])


In [ ]:
#T-test Validation for RFF
i = 1
for x in range(0, len(ttest_pvals)):
    for y in range(0, len(ttest_pvals[x])):
        if ttest_pvals[x][y] < 0.05:
            print("H0 Rejected. Component " + str(i) + " has significant difference to the other components.")
            break
        elif y >= len(ttest_pvals[x]) - 1:
            print("H0 Accepted. Component " + str(i) + " has no significant difference to the other components.")
    i = i + 1
        

In [ ]:
#F-test Validation for RFF

for x in range(0, len(p_value_f)):
    if p_value_f[x] < 0.05:
        print("H0 Rejected. Components have significant differences ")
        break
    elif x >= len(p_value_f) - 1:
        print("H0 Accepted. Components don't have significant differences")
    

In [ ]:
#Random Forest Values Backward
#t-test for one vs all

classes = ['PRAD', 'LUAD', 'BRCA', 'KIRC', 'COAD']
ttest_pvals = []

for x in rfb_df[gene_names].columns:
    gene = rfb_df[x]
    gene_p_val = []
    for y in classes:
        group1_expression = rfb_df[gene_names[x]][rfb_df['Class'] == y] 
        group2_expression = rfb_df[gene_names[x]][rfb_df['Class'] != y]  
        _, p_value = ttest_ind(group1_expression, group2_expression)
        gene_p_val.append(p_value)
    ttest_pvals.append(gene_p_val)

#F-test

_, p_value_f = f_oneway(rfb_df[gene_names][rfb_df['Class'] == 'PRAD'], rfb_df[gene_names][rfb_df['Class'] == 'LUAD'], rfb_df[gene_names][rfb_df['Class'] == 'BRCA'], rfb_df[gene_names][rfb_df['Class'] == 'KIRC'], rfb_df[gene_names][rfb_df['Class'] == 'COAD'])



In [ ]:
#T-test Validation for RFB
i = 1
for x in range(0, len(ttest_pvals)):
    for y in range(0, len(ttest_pvals[x])):
        if ttest_pvals[x][y] < 0.05:
            print("H0 Rejected. Component " + str(i) + " has significant difference to the other components.")
            break
        elif y >= len(ttest_pvals[x]) - 1:
            print("H0 Accepted. Component " + str(i) + " has no significant difference to the other components.")
    i = i + 1

In [ ]:
#F-test Validation for RBF

for x in range(0, len(p_value_f)):
    if p_value_f[x] < 0.05:
        print("H0 Rejected. Components have significant differences ")
        break
    elif x >= len(p_value_f) - 1:
        print("H0 Accepted. Components don't have significant differences")

In [ ]:
#SVM Forward
#t-test for one vs all

classes = ['PRAD', 'LUAD', 'BRCA', 'KIRC', 'COAD']
ttest_pvals = []

for x in svcf_df[gene_names].columns:
    gene = svcf_df[x]
    gene_p_val = []
    for y in classes:
        group1_expression = svcf_df[gene_names[x]][svcf_df['Class'] == y] 
        group2_expression = svcf_df[gene_names[x]][svcf_df['Class'] != y]  
        _, p_value = ttest_ind(group1_expression, group2_expression)
        gene_p_val.append(p_value)
    ttest_pvals.append(gene_p_val)

#F-test

_, p_value_f = f_oneway(svcf_df[gene_names][svcf_df['Class'] == 'PRAD'], svcf_df[gene_names][svcf_df['Class'] == 'LUAD'], svcf_df[gene_names][svcf_df['Class'] == 'BRCA'], svcf_df[gene_names][svcf_df['Class'] == 'KIRC'], svcf_df[gene_names][svcf_df['Class'] == 'COAD'])



In [ ]:
#T-test Validation for SVCF
i = 1
for x in range(0, len(ttest_pvals)):
    for y in range(0, len(ttest_pvals[x])):
        if ttest_pvals[x][y] < 0.05:
            print("H0 Rejected. Component " + str(i) + " has significant difference to the other components.")
            break
        elif y >= len(ttest_pvals[x]) - 1:
            print("H0 Accepted. Component " + str(i) + " has no significant difference to the other components.")
    i = i + 1

In [ ]:
#F-test Validation for SVCF
for x in range(0, len(p_value_f)):
    if p_value_f[x] < 0.05:
        print("H0 Rejected. Components have significant differences ")
        break
    elif x >= len(p_value_f) - 1:
        print("H0 Accepted. Components don't have significant differences")